In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
import scipy
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from sklearn import metrics
import itertools
from sklearn.decomposition import PCA
from scipy.sparse import csgraph
import umap
from collections import Counter
import sklearn
from importlib import reload

import gget

# locals
import utils as ut
reload(ut)

2023-06-21 16:51:07.228871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 16:51:14.612897: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 16:51:14.612921: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-21 16:51:15.341512: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-21 16:51:24.006129: W tensorflow/stream_executor/pla

<module 'utils' from '/home/cstansbu/git_repositories/spatial_transcriptomics/clustering/utils.py'>

In [2]:
cardDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"
xyPath =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/combinedEmbedding.csv"
xy = pd.read_csv(xyPath)
cTypes = sorted(list(xy['cellTypes'].unique()))
print(cTypes)
print()

keys = ['ND', 'HFD8', 'HFD14']

rna = {}
labels = {}

for key in keys:
    print(f"{key}...")
    labelPath = f"{cardDir}{key}_macrophage_clusters.csv"
    lf = pd.read_csv(labelPath)
    
    lf = pd.merge(lf, xy[['x', 'y', 'cellId', 'colors']], 
                  how='left',
                  left_on='cellId',
                  right_on='cellId')
    
    labels[key] = lf
    
    rnaPath = f"{cardDir}{key}_macrophage_rna.csv"
    rf = pd.read_csv(rnaPath)
    rf = rf.T
    new_header = rf.iloc[0] 
    rf = rf[1:] 
    rf.columns = new_header 
    rf.index.names = ['cellId']
    
    rf = ut.normalize(rf, 1e6)
    
    rna[key] = rf

print('done')

['B cells', 'Dendritic cells', 'Macrophages', 'Monocytes', 'NK cells', 'Stromal cells', 'T cells']

ND...
HFD8...
HFD14...
done


In [3]:
genes = [x.strip().upper() for x in open("./lindseyGenes.txt")]
genes = [x for x in genes if x in rna['ND'].columns]
len(genes)

1528

In [4]:
outdir = "~/temp/"

for key in keys:
    rf = rna[key]
    lf = labels[key]


    pdf = pd.merge(rf[genes], lf[['cellId', 'cellType']],
                   how='left',
                   left_index=True,
                   right_on='cellId')

    print(f"{key} {rf.shape=} {lf.shape=} {pdf.shape=}")

    del rf
    del lf
    print(f"{key} done merge")

    fpath = f"{outdir}{key}_CPM.csv"
    pdf.to_csv(fpath, index=False)
    print(f'{key} done')

    # print(pdf.head())
    # break




ND rf.shape=(1261, 31053) lf.shape=(1265, 6) pdf.shape=(1265, 1530)
ND done merge
ND done
HFD8 rf.shape=(6123, 31053) lf.shape=(6136, 6) pdf.shape=(6136, 1530)
HFD8 done merge
HFD8 done
HFD14 rf.shape=(6436, 31053) lf.shape=(6453, 6) pdf.shape=(6453, 1530)
HFD14 done merge
HFD14 done


In [5]:
print('done')

done
